In [1]:
from cartes.osm import Overpass
from shapely.ops import linemerge
import altair as alt
import pandas as pd
from palettable.tableau import Tableau_20
import numpy as np
import geopandas
import gpdvega
from cartes.crs import EuroPP

alt.themes.enable("opaque")


ERROR 1: PROJ: proj_create_from_database: Open of /home/marcel/.conda/envs/osm/share/proj failed


ThemeRegistry.enable('opaque')

In [2]:
# area_name = "Schleswig-Holstein"
# top_admin_level = "4"
# parts_admin_level = "[6]"
# buffer = 0.001
# train = True
width = 1920
height = 1080
# columns = 2
top_admin_level = "6"
parts_admin_level = "[10]"
area_name = "Kiel"
buffer = 0.005
train = False
columns = 3
filename = area_name

In [3]:
from cartes.osm.overpass.core import Relation

import itertools
from operator import itemgetter

from shapely.geometry.multilinestring import MultiLineString
from shapely.geometry.polygon import Polygon
from shapely.geometry.linestring import LineString
from shapely.ops import linemerge, unary_union

from cartes.osm.overpass import Overpass


class Route(Relation):
    """A class to parse route=train relations.
    The purpose of train route relations is to have an object for each train route.
    Reference: https://wiki.openstreetmap.org/wiki/Tag:route%3Dtrain
    Tags:
      - type (route)
      - route (train)

    Relation members:
      - None or main_stream 1+
      - side_stream 0+
      - spring ?
      - tributary 0+
    """

    def __init__(self, json):
        super().__init__(json)

        self.parent: Overpass = json["_parent"]
        parsed_keys = dict(
            (elt["ref"], elt) for elt in self.parent.all_members[json["id_"]]
        )

        parts = dict(
            (
                role,
                unary_union(
                    list(elt["geometry"] for elt in it if elt.get("geometry", None))
                ),
            )
            for role, it in itertools.groupby(
                parsed_keys.values(), key=itemgetter("role")
            )
        )
        ways = parts.get("", None)
        elements = [linemerge(ways) if isinstance(ways, MultiLineString) else ways]
        stops = parts.get("stop", None)
        if stops is not None:
            elements.append(
                linemerge(stops) if isinstance(stops, MultiLineString) else stops
            )
        platforms = parts.get("platform", None)
        if platforms is not None:
            elements.append(
                linemerge(platforms)
                if isinstance(platforms, MultiLineString)
                else platforms
            )

        list_ = list(elt for elt in elements if elt is not None)
        if len(list_) == 0:
            self.json["geometry"] = self.shape = None
        self.json["geometry"] = self.shape = unary_union(list_)


In [4]:
sh_area = Overpass.request(
    area={"name": area_name, "admin_level": top_admin_level},
    rel=dict(boundary="administrative", admin_level=dict(regex=f"{parts_admin_level}")),
)
# Overpass.build_query(area={"name": area_name, "admin_level": top_admin_level},
#     rel=dict(boundary="administrative", admin_level=dict(regex=f"{parts_admin_level}")))



In [5]:
colors = [
    "#d9328c",
    "#5bc2ef",
    "#85c442",
    "#9e1b2a",
    "#853b93",
    "#0f7c3f",
    "#e41d25",
    "#f4892e",
    "#fcc12c",
    "#836cae",
    "#1963b0",
    "#2e3147",
    "#8aba54",
    "#8c6095",
    "#1ab26b",
    "#cb4b36",
    "#e9b651",
    "#de904a",
]


In [6]:
routes = [
        dict(route="train", area="sh"),  # train lines
        dict(public_transport="station", area="sh", train="yes"),  # train stations
    ] if train else [
        dict(route="bus", area="sh"),  # bus lines
        dict(public_transport="station", area="sh", bus="yes"),  # bus stations
    ]

sh_trains = Overpass.request(
    area={"name": area_name, "admin_level": top_admin_level, "as_": "sh"},
    nwr=routes,
)

print(Overpass.build_query(
    area={"name": area_name, "admin_level": top_admin_level, "as_": "sh"},
    nwr=[
        dict(route="train", area="sh"),  # train lines
        dict(public_transport="station", area="sh", train="yes"),  # train stations
    ],))


lines = sh_trains.data.query("type_ == 'relation'")

[out:json][timeout:180];area[name=Kiel][admin_level=6]->.sh;nwr(area.sh)[route=train];out geom;nwr(area.sh)[public_transport=station][train=yes];out geom;


In [7]:
lines = lines[["name", "geometry", "ref"]]


def fix_types(elt):
    if elt.geom_type == "Polygon":
        return elt.exterior
    else:
        return elt


def merge_line(elt):
    return pd.Series(
        {
            "geometry": linemerge(elt.geometry.tolist()),
            "name": elt["name"].max(),
        }
    )


lines["geometry"] = lines["geometry"].apply(fix_types)
lines = lines.explode(index_parts=True)
lines["name"] = lines["name"].fillna("Andere")
lines["name"] = (
    lines["name"].replace("<=", "<=>", regex=True).replace("=>", "<=>", regex=True)
)

lines = (
    lines.groupby("ref")
    .apply(merge_line)
    .reset_index(level=0, drop=True)
    .reset_index()
    .set_index("index")
)


In [8]:
i = 0


def setColor():
    global i
    color = colors[i % len(colors)]
    i = i + 1
    return color


if "colour" not in lines:
    lines["colour"] = lines.apply(lambda x: setColor(), axis=1)
else:
    lines["colour"] = np.where(lines["colour"] == "", setColor(), lines["colour"])
colours = (
    lines[["name", "colour"]]
    .query("colour==colour")
    .groupby("name")
    .agg({"name": "max", "colour": "max"})
    .reset_index(level=0, drop=True)
    .reset_index()
    .set_index("index")
)

lines.to_file(f'{filename}.geojson', driver='GeoJSON')  


In [9]:
# First the colors
line_scale = alt.Scale(
    domain=colours["name"].tolist(), range=colours["colour"].tolist()
)

wards = alt.Chart(sh_area)


projection = EuroPP()
basemap = (
    alt.layer(
        # The background
        wards.mark_geoshape(color="#DCDDDD", stroke="white", strokeWidth=1),
        # The names of the wards: In german
        wards.mark_text(fontSize=16, font="Noto Sans", fontWeight=100).encode(
            alt.Text("name:N"),
            alt.Latitude("latitude:Q"),
            alt.Longitude("longitude:Q"),
        ),
        # The railway lines: in In german
        alt.Chart(lines.extent(area_name, projection, buffer=buffer))
        .mark_geoshape(filled=False, strokeWidth=2)
        .encode(
            alt.Color(
                "name:N",
                scale=line_scale,
                legend=alt.Legend(
                    title="Linien",
                    orient="bottom-left",
                    columns=columns,
                    symbolLimit=0,
                ),
            ),
            alt.Tooltip(["name:N"]),
        ),
        # railway stations positions
        alt.Chart(
            sh_trains.query("public_transport == 'station'").data
        )
        .mark_circle(size=30, color="darkslategray")
        .encode(
            alt.Text("name:N"),
            alt.Latitude("latitude:Q"),
            alt.Longitude("longitude:Q"),
            alt.Tooltip(["name:N"]),
        ),
    )
    .properties(width=width, height=height)
    .configure_view(strokeWidth=0)
)

basemap


/home/marcel/.conda/envs/osm/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():
/home/marcel/.conda/envs/osm/lib/python3.9/site-packages/gpdvega/geodata.py:38: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  data.drop(
/home/marcel/.conda/envs/osm/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():
/home/marcel/.conda/envs/osm/lib/python3.9/site-packages/gpdvega/geodata.py:38: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a fut

alt.LayerChart(...)